## Design Patterns with First-Class Functions

In [1]:
# Example 6-3, refactoring the "Strategy" example using a function-oriented approach

In [2]:
from collections import namedtuple

In [3]:
Customer = namedtuple('Customer', 'name fidelity')

In [4]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.10
    return discount

def large_order_promo(order):
    """7% discount for order with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [5]:
# sample usage
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [7]:
# Pass the promotion function as an argument
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [8]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [9]:
banana_cart = [LineItem('banana', 30, 0.5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [10]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [12]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

### Example 6-6 choosing the best strategy: a simple approach

In [14]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
# promos is a LIST of Functions

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

In [15]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [16]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [17]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [ ]:
globals() # Return a dictionary representing the global symbol table